# 6_serve_model_with_Flask

In this notebook, we'll make a [Flask](https://www.fullstackpython.com/flask.html) web form application to show how our models can interact with a web application.

## Set up environment variables and load necessary libraries

In [ ]:
import os
PROJECT = "predict-babyweight-10142021"
BUCKET = PROJECT
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET 
os.environ["REGION"] = REGION

## Step 1: Review Flask App code in `application` folder
Let's start with what our users will see. In the `application` folder, we have prebuilt the components for web application. In the templates folder, the <a href="application/templates/index.html">index.html</a> file is the visual GUI our users will make predictions with.

It works by using an HTML [form](https://www.w3schools.com/html/html_forms.asp) to make a [POST request](https://www.w3schools.com/tags/ref_httpmethods.asp) to our server, passing along the values captured by the [input tags](https://www.w3schools.com/html/html_form_input_types.asp).

The form will render a little strangely in the notebook since the notebook environment does not run javascript, nor do we have our web server up and running. Let's get to that!

## Step 3: Complete application code in `application/main.py`
We can set up our server with python using [Flask](https://flask.palletsprojects.com/en/1.1.x/quickstart/). Below, we've already built out most of the application for you.

The `@app.route()` decorator defines a function to handle web reqests. Let's say our website is `www.example.com`. With how our `@app.route("/")` function is defined, our sever will render our <a href="application/templates/index.html">index.html</a> file when users go to `www.example.com/` (which is the default route for a website).

So, when a user pings our server with `www.example.com/predict`, they would use `@app.route("/predict", methods=["POST"])` to make a prediction. The data that gets sent over the internet isn't a dictionary, but a string like below:

`name1=value1&name2=value2` where `name` corresponds to the `name` on the input tag of our html form, and the value is what the user entered. Thankfully, Flask makes it easy to transform this string into a dictionary with `request.form.to_dict()`, but we still need to transform the data into a format our model expects. We've done this with the `gender2str` and the `plurality2str` utility functions.

Ok! Let's set up a webserver to take in the form inputs, process them into features, and send these features to our model on Cloud AI Platform to generate predictions to serve to back to users.

Fill in the **TODO** comments in <a href="application/main.py">application/main.py</a>. Give it a go first and review the solutions folder if you get stuck.

**Note:** AppEngine test configurations have already been set for you in the file <a href="application/app.yaml">application/app.yaml</a>. Review [app.yaml](https://cloud.google.com/appengine/docs/standard/python/config/appref) documentation for additional configuration options.

## Step 4: Deploy application
So how do we know that it works? We'll have to deploy our website and find out! Notebooks aren't made for website deployment, so we'll move our operation to the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true).

By default, the shell doesn't have Flask installed, so copy over the following command to install it.

`python3 -m pip install --user Flask==0.12.1`

Next, we'll need to copy our web app to the Cloud Shell. We can use [Google Cloud Storage](https://cloud.google.com/storage) as an inbetween.

In [ ]:
%%bash
# Deploy a web application that consumes your model service on Cloud AI Platform
gsutil -m rm -r gs://$BUCKET/prediction_app
gsutil -m cp -r application/ gs://$BUCKET/prediction_app

Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/Icon-checkpoint#1635001996151865...
Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/app-checkpoint.yaml#1635001996168750...
Removing gs://predict-babyweight-10142021/prediction_app/.ipynb_checkpoints/main-checkpoint.py#1635001996061983...
Removing gs://predict-babyweight-10142021/prediction_app/Icon#1635001995963341...
Removing gs://predict-babyweight-10142021/prediction_app/main.py#1635001995982540...
Removing gs://predict-babyweight-10142021/prediction_app/app.yaml#1635001995948000...
Removing gs://predict-babyweight-10142021/prediction_app/static/.ipynb_checkpoints/baby-checkpoint.png#1635001996177767...
Removing gs://predict-babyweight-10142021/prediction_app/static/.ipynb_checkpoints/feet-checkpoint.png#1635001996130233...
Removing gs://predict-babyweight-10142021/prediction_app/static/feet.png#1635001995996596...
Removing gs://predict-babyweight-10142021/prediction_app/template

Run the below cell, and copy the output into the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true)

In [ ]:
%%bash
echo rm -r prediction_app/
echo mkdir prediction_app/
echo gsutil cp -r gs://$BUCKET/prediction_app ./
echo python3 prediction_app/main.py

rm -r prediction_app/
mkdir prediction_app/
gsutil cp -r gs://predict-babyweight-10142021/prediction_app ./
python3 prediction_app/main.py


## Step 5: Use your website to generate predictions
Time to play with the website! The cloud shell should now say something like `* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)`. Click on the `http` link to go to your shiny new website. Fill out the form and give it a minute or two to process its first prediction. After the first one, the rest of the predictions will be lightning fast.

Did you get a prediction? If not, the Google Cloud Shell will spit out a stack trace of the error to help narrow it down. If yes, congratulations! Great job on bringing all of your work together for the users.

## Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by running the following
commands:

In [ ]:
## Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by running the following
commands:

In [ ]:
# Delete model version resource
! gcloud ai-platform versions delete $MODEL_VERSION --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# If the training job is still running, cancel it
! gcloud ai-platform jobs cancel $JOB_NAME --quiet --verbosity critical

If your Cloud Storage bucket doesn't contain any other objects and you would like to delete it, run gsutil rm -r gs://$BUCKET_NAME.

## Lab Summary



In this lab, you deployed a simple Flask web form application on App Engine that takes inputs, transforms them into features, and sends them to a model service on Cloud AI Platform to generate and return predictions.